In [36]:

# Unsafe Connection
from sqlalchemy import create_engine, text
import pandas as pd

server = "OOO"
database = "bokhandel"
driver = "ODBC Driver 17 for SQL Server"

engine = create_engine(f"mssql+pyodbc://{server}/{database}?Trusted_Connection=yes&driver={driver}")

In [ ]:

# Safe User Based Connection
from sqlalchemy import create_engine, text
import pandas as pd

server = "OOO"
database = "bokhandel"
driver = "ODBC Driver 17 for SQL Server"
user = 'new_user' 
password = 'abc'

engine = create_engine(f"mssql+pyodbc://{user}:{password}@{server}/{database}?driver={driver}")

In [13]:
with engine.connect() as connection:
    pass

In [ ]:
query = "DROP TABLE LagerSaldo;"
with engine.connect() as connection:
    connection.execute(text(query))
    connection.commit()

ProgrammingError: (pyodbc.ProgrammingError) ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Cannot drop the table 'LagerSaldo', because it does not exist or you do not have permission. (3701) (SQLExecDirectW)")
[SQL: DROP TABLE LagerSaldo;]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
# Unsafe Query, if you have a safe user based connection where the user is 
# restricted to SELECt then this whould be safe 
user_input = "bobs bookshop'; DROP TABLE LagerSaldo --"
query = f""" 
SELECT ButiksNamn
FROM [Butiker] 
WHERE ButiksNamn LIKE '%{user_input}%'
"""

with engine.connect() as connection:
    result = connection.execute(text(query))
    connection.commit()

In [ ]:
# Safe Query, this could be used with the unsafe connection and 
# still prevent the SQL injection because SQL Alchemy handels the 
# string sanitization  
user_input = "bobs bookshop'; DROP TABLE LagerSaldo --"
query = f""" 
SELECT ButiksNamn
FROM [Butiker] 
WHERE ButiksNamn LIKE '%:user_value%'
"""

with engine.connect() as connection:
    result = connection.execute(text(query), {"user_vlaue": user_input})
    connection.commit()

In [ ]:
with engine.connect() as connection:
    # Use a named parameter (e.g., :value)
    query_template = text("INSERT INTO MyTable (MyColumn) VALUES (:user_input_param)")
    # Pass the user input as a dictionary mapping parameter names to values
    connection.execute(query_template, {"user_input_param": user_value})
    connection.commit()

In [17]:
user_input = input()
query = f""" 
SELECT ButiksNamn, Title, Antal
FROM [Butiker] 
JOIN [LagerSaldo] ON [Butiker].[ID] = [LagerSaldo].[ButikID]
JOIN [Böcker] ON [LagerSaldo].[ISBN13] = [Böcker].[ISBN13]
WHERE Title LIKE '%{user_input}%'
"""

query_2 = """ 
DROP TABLE LagerSaldo
"""

with engine.connect() as connection:
    # connection = connection.execution_options(
    #     isolation_level = "SERIALIZABLE",
    #     postgresql_readonly=True, 
    #     postgresql_deferrable=True 
    # )
    
    # Using a raw SQL query with text()
    
    result = connection.execute(text(query))
    
    
    df = pd.DataFrame(data=result, columns=["Butik", "Boktitle", "Exemplar"] )
    
    #for row in result: 
    #    print(row)
df


,Butik,Boktitle,Exemplar
0,The Fox,A Midsummer Day,10
1,The Hound,The Drive West,3
2,The Hound,Nightmare Friday,8
3,The Hound,A Midsummer Day,0
4,The Hound,Grimhaven,9
5,The Goose,The Big Smile,4
6,The Badger,A Field of Cannons,21
7,The Badger,The Drive West,1
8,The Crane,The Big Smile,5
9,The Crane,Nightmare Friday,2


# TABELS and Columns

### Tabel: Böcker
Besides PRIMARY KEY the CONSTRAINT on ISBN13 where set to
    CHECK ([ISBN13] LIKE '___-_-___-_____-_') 
Which makes sure the length and formating of the ISBN13 string
is correct thanks to the _ wildcard, howerver it can safegaurd against putting in letters instead of numbers. 
Two more colums where added, 'AntalSidor' aka how many pages there is the book and 'Format' of the book ex pocket or bunden

### Table : Inköp 
This is one of four new tables that contains the order that the bookstores have placed. It contains data such as the amount of books when the books where ordered and when there excpected to arrive. This seems like a usefull table for stores to have to keep track of orders in progress as well as being able to refere back to past orders. 

### Tables: Anstälda & Roll
Anstälda is the secnond new table, it contains the contact information for the employes that working at the stores, it also linked to the third new table that contains the rolls that the employes takes on, in this case the rolls is a seperate table allowing for the personal to have multiple jobs like 'Inköpare' och 'Butiksägare'. 
These can be good for a bookshop to have so one can easily find the contact information for the employees and if you want to contact another store the roller will help you find the right person in an other store.

### Table: Genre 
This is the forth table and it allows one book to have multiple genres and it can make it easier for the store to check what type of genres sell the best.


# VIEW 

### TitlarPerFörfatter
Lagerantal was addad to give some more context to Lagervärde.
There might have been a better solution but because Namn used an Alias that wasen't created untill after the GROUP BY part had been executed and Namn needed to be either aggregated or added to the GROUP BY else the query wouldn't have worked. The soluntion was to use CROSS APPLY so that the Alias could be declared before the GROUP BY and thus could be used with the GROUP BY.


In [44]:
query = "SELECT TOP 1 * FROM TitlarPerFörfattare"
with engine.connect() as connection:
    result = connection.execute(text(query))
    df = pd.DataFrame(data=result, columns=["Namn", "Ålder", "Titlar", "Lagerantal", "Lagervärde"] )
df

,Namn,Ålder,Titlar,Lagerantal,Lagervärde
0,Will Russel,70 År,4 st,66 st,3419 kr


### Personalinköp
This View keeps tracks on how many books a employ has order and the total value of the books. This could be an intresting statistic to keep track of, it can indicate when a store needs to hire a person dedicated to placing orders or if coupled with how well books have sold this could indicate conditions for a promotion.  

In [45]:
query = "SELECT TOP 5 * FROM Personalinköp "
with engine.connect() as connection:
    result = connection.execute(text(query))
    df = pd.DataFrame(data=result, columns=["Namn", "Antal Bestälda Böcker", "Totatlt Bestälnings Värde"] )
df

,Namn,Antal Bestälda Böcker,Totatlt Bestälnings Värde
0,David Svensson,16 st,984 kr
1,Erik Bergman,13 st,599 kr
2,Fatima Khan,28 st,2503 kr
3,Johana Strong,98 st,6298 kr
4,Julia Eriksson,49 st,4118 kr


# Stored Porcedure


I used an LLM `Geini 2.0` to generate the TRY Catch and Error part of the Store Procedure

    I have this t-sql code and I have a CONSTRAINT on Antal, 
    Antal can never be lower then 0. I'm using BEGIN TRANSACTION to try 
    and prevent the first UPDATE to execute unless the second UPDATE also 
    gets executed but it doesn't work, the second UPDATE fail and the 
    first UPDATE stil gets executed
    ` CODE `

    ...

    Gem I have this t-sql stored procedure, it works but the problem is 
    that it doesn't rollback when one of the updates effekts zero rows
    ` CODE `


The user can't execute this Stored Procedure because it alters tthe tables of the database.
I should give the user more access or use a differnt user account, but I'm lazy and so 
I am going with the Unsafe Windows Authentication Connection.

In [61]:
# Unsafe Connection
from sqlalchemy import create_engine, text
import pandas as pd

server = "OOO"
database = "bokhandel"
driver = "ODBC Driver 17 for SQL Server"

engine = create_engine(f"mssql+pyodbc://{server}/{database}?Trusted_Connection=yes&driver={driver}")

In [ ]:
stored_procedure = "EXEC FlyttaBok @från = 1, @till = 2, @isbn = '978-0-679-72981-7';"
query = "SELECT * FROM [LagerSaldo] WHERE [ISBN13] = '978-0-679-72981-7';"

with engine.connect() as connection:
    connection.execute(text(stored_procedure))
    connection.commit()
    result = connection.execute(text(query)) 
    df = pd.DataFrame(data=result, columns=["ButikID", "ISBN13", "Antal"] )
df

,ButikID,ISBN13,Antal
0,1,978-0-679-72981-7,9
1,2,978-0-679-72981-7,6
